In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Extract data
df_tr = pd.read_csv('Data\Intrusion-Detection\Train_data.csv',na_values='?',header=0)
df_tr.head(6);

In [5]:
df_ts = pd.read_csv('Data\Intrusion-Detection\Test_data.csv',na_values='?',header=0)
df_ts.head(6);

In [6]:
#Drop ClassLabel column from input array X and Create output array y of Classlabel.
Xtr = np.array(df_tr.drop('xAttack',axis=1))
ytr = np.array(df_tr['xAttack'])
Xts = np.array(df_ts.drop('xAttack',axis=1))
yts = np.array(df_ts['xAttack'])

In [7]:
#Encode
le = preprocessing.LabelEncoder()
lb = preprocessing.LabelBinarizer()

In [8]:
Xtr[:,1] = le.fit_transform(Xtr[:,1])
Xts[:,2] = le.fit_transform(Xts[:,2])
Xts = np.delete(Xts,0,axis=1)


ytr = le.fit_transform(ytr)
lb.fit_transform(ytr)

yts = le.fit_transform(yts)
lb.fit_transform(yts);


In [9]:
#Nomralize the data
Xtr_n = preprocessing.normalize(Xtr)
Xts_n = preprocessing.normalize(Xts)

In [10]:
from sklearn.decomposition import PCA
ncomp =41
# Construct the PCA object
pca = PCA(n_components=ncomp, 
          svd_solver='randomized', whiten=True)

# Fit the PCA components on the entire dataset
pca.fit(Xtr_n)
Ztr = pca.transform(Xtr_n)
Zts = pca.transform(Xts_n)

In [12]:
S = pca.singular_values_

lam = S**2
PoV = np.cumsum(lam)/np.sum(lam)

plt.plot(PoV)
plt.xlabel('No. of PCs')
plt.ylabel('PoV')
plt.grid()

In [13]:
ncomp =10
# Construct the PCA object
pca = PCA(n_components=ncomp, 
          svd_solver='randomized', whiten=True)

# Fit the PCA components on the entire dataset
pca.fit(Xtr_n)
Ztr = pca.transform(Xtr_n)
Zts = pca.transform(Xts_n)

In [14]:
K.clear_session()

In [15]:
nin = Ztr.shape[1]  # dimension of input data
nh = 100     # number of hidden units
nout = int(np.max(ytr)+1)    # number of outputs = 10 since there are 10 classes
model = Sequential()
model.add(Dense(units=nh, input_shape=(nin,), activation='sigmoid', name='hidden1'))
#model.add(Dense(units=50, activation='sigmoid', name='hidden2'))
model.add(Dense(units=nout, activation='softmax', name='output'))

In [19]:
model.summary()

In [21]:
from tensorflow.keras import optimizers

opt = optimizers.Adam(lr=0.001) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
hist = model.fit(Ztr, ytr, epochs=30, batch_size=100, validation_data=(Zts,yts))

In [24]:
tr_accuracy = hist.history['acc']
val_accuracy = hist.history['val_acc']

plt.plot(tr_accuracy)
plt.plot(val_accuracy)
plt.grid()
plt.xlabel('epochs')
plt.ylabel('accuarcy')
plt.legend(['training accuracy', 'validation accuracy'])

In [28]:
score, acc = model.evaluate(Zts, yts, verbose=1)
print("accuracy = %f" % acc)

In [30]:
maxTr = np.argmax(tr_accuracy)
maxTs = np.argmax(val_accuracy)
print('Max Training Accuracy obtained:',tr_accuracy[maxTr])
print('Max Test Accuracy obtained:',val_accuracy[maxTs])